In [2]:
import os
import json
import sys

In [3]:
with open("benchmark.txt", "r") as f:
    lines = f.readlines()

count = 0
nonqa_count = 0
sum = 0
qa_set = []
nonqa_set = []

for line in lines:
    if ('-A' in line):
        qa_set.append(line)
        count += 1
    else:
        nonqa_set.append(line)
        nonqa_count += 1

print(count, nonqa_count)

# with open("qa_set.txt", "w", encoding='utf-8') as f:
#     json.dump(qa_set, f, ensure_ascii=False)

# with open("nonqa_set.txt", "w", encoding='utf-8') as f:
#     json.dump(nonqa_set, f, ensure_ascii=False)




108 409


In [3]:
nonqa_set = json.load(open("nonqa_set.txt", "r"))

In [4]:
nonqa_set = [line.strip() for line in nonqa_set]

In [6]:
len(nonqa_set)

409

In [57]:
# q_count = 0
# clean_nonqa_set = []
# tmp = ''
# for i in range(len(nonqa_set)):
#     if "\\" in nonqa_set[i]:
#         tmp = nonqa_set[i].split("\\")[-1]
#         clean_nonqa_set.append(tmp)
#         tmp = ''
#     else:
#         tmp = tmp + nonqa_set[i]
#         print(tmp)

# nonqa_set

In [34]:
clean_nonqa_set[-1]

'Q：Which is more important for dielectric repulsion, the image force or the Born effect?'

In [12]:
import deepeval
deepeval.login_with_confident_api_key("od9egJUwzwFdCJ6kzxMCrEWpZVsVZtdxsUETSxjausk=")

Congratulations! Login successful 🙌

In [9]:
from deepeval.dataset import EvaluationDataset

/Users/masa/miniforge3/envs/torch/lib/python3.8/site-packages/deepeval/__init__.py:51: UserWarning: You are using deepeval version 2.0, however version 2.6.5 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [37]:
# dataset = EvaluationDataset()
# dataset.pull(alias="MembraneTest")

In [40]:
full_nonqa_set = ''
for i in range(len(nonqa_set)):
    if i==0:
        pass
    else:
        full_nonqa_set += nonqa_set[i]

In [42]:
# Split the full_nonqa_set string into individual questions
def split_questions(text):
    # Use regex to split by pattern like "digits\Q:"
    questions = re.split(r'(\d+\\Q:)', text)
    
    # The split will create empty first element if text starts with the pattern
    # and will separate the pattern from the content
    result = []
    
    # Skip first element if it's empty
    start_idx = 0
    if questions[0] == '':
        start_idx = 1
    
    # Combine the pattern with its content
    for i in range(start_idx, len(questions), 2):
        if i+1 < len(questions):
            result.append(questions[i] + questions[i+1])
    
    return result

# Apply the function to split the questions
split_nonqa_set = split_questions(full_nonqa_set)

# Print the first few questions to verify
for i, q in enumerate(split_nonqa_set[:3]):
    print(f"Question {i+1}:\n{q}\n")

# Print the total number of questions
print(f"Total questions: {len(split_nonqa_set)}")

Question 1:
1\Q:请用一句话阐述相转化膜的定义。 (通过适当途径使聚合物从均相铸膜液中沉析，形成聚合物富相和聚合物贫相的膜) (GB/T 20103-2006)

Question 2:
3\Q:请用一句话阐述复合膜的定义。 (用两种不同的膜材料，分别制成具有分离功能的表面活性层和起支撑作用的多孔层组成的膜) (GB/T 20103-2006)

Question 3:
5\Q:请用一句话阐述膜元件和膜组件的关系。 (膜组件是由膜元件、壳体、内联接件、端板和密封圈等组成的实用器件) (GB/T 20103-2006)

Total questions: 123


In [43]:
split_nonqa_set

['1\\Q:请用一句话阐述相转化膜的定义。 (通过适当途径使聚合物从均相铸膜液中沉析，形成聚合物富相和聚合物贫相的膜) (GB/T 20103-2006)',
 '3\\Q:请用一句话阐述复合膜的定义。 (用两种不同的膜材料，分别制成具有分离功能的表面活性层和起支撑作用的多孔层组成的膜) (GB/T 20103-2006)',
 '5\\Q:请用一句话阐述膜元件和膜组件的关系。 (膜组件是由膜元件、壳体、内联接件、端板和密封圈等组成的实用器件) (GB/T 20103-2006)',
 '7\\Q:请用一句话阐述错流膜过程。 (压力推动给料平行于膜表面流动，而透过液垂直透过膜的分离过程) (GB/T 20103-2006)',
 '8\\Q:请有一句话分别阐述膜装置流程中段和级的概念。 (给水每流经一组膜组件为一段；给水每流经由增压泵和膜组件等组成的一个系统为一级) (GB/T 20103-2006)',
 '16\\Q: How to confirm the successful incorporation of nanoparticles?Context: Both TFC and TFN membranes exhibit nano-scale surface roughness, which is a well-known characteristic of interfacial polymerized polyamide RO membranes. The irregular morphology precludes quantification of a single film layer thickness, but both thin films are about 50–200 nm. In Fig. 3(c), zeolite nanoparticles appear considerably darker than the polymer and are clearly located within the cross-section of the thin film, but also at the interface.Key: In TEM images, they can be shown in a darker color with

In [49]:
import pandas as pd
import re
import json

def parse_questions(question_list):
    data = []
    
    # First combine items belonging to the same question
    grouped_questions = []
    current_group = []
    
    for item in question_list:
        if '\\Q:' in item:  # Start of a new question
            if current_group:  # Save previous group if exists
                grouped_questions.append('\n'.join(current_group))
            current_group = [item]
        else:
            current_group.append(item)
    
    # Don't forget to add the last group
    if current_group:
        grouped_questions.append('\n'.join(current_group))
    
    # Now process each grouped question
    for question_text in grouped_questions:
        lines = question_text.split('\n')
        
        # Initialize variables
        question_number = None
        question = None
        context = None
        expected_output = None
        retrieval_context = None
        
        # Extract question number and text from the first line
        first_line = lines[0]
        match = re.match(r'(\d+)\\Q:(.*)', first_line)
        if match:
            question_number = match.group(1)
            question = match.group(2).strip()
        
        # Single line format processing
        if len(lines) == 1:
            # Find all text in parentheses
            parentheses_matches = re.findall(r'\((.*?)\)', first_line)
            
            if len(parentheses_matches) >= 2:
                # The last parenthesis is the retrieval context
                retrieval_context = parentheses_matches[-1].strip()
                
                # The second-to-last parenthesis (or first if only two) is the expected output
                expected_output = parentheses_matches[-2].strip()
                
                # Remove parentheses content from question
                for match in parentheses_matches:
                    question = question.replace(f"({match})", "").strip()
            elif len(parentheses_matches) == 1:
                # If there's only one parenthesis, assume it's the expected output
                expected_output = parentheses_matches[0].strip()
                question = question.replace(f"({expected_output})", "").strip()
        
        # Multi-line format processing
        else:
            # Look for Context line
            context_text = ""
            for i, line in enumerate(lines):
                if line.startswith('Context:'):
                    context_text = line.replace('Context:', '').strip()
                    # Check if context continues on next lines
                    j = i + 1
                    while j < len(lines) and not (lines[j].startswith('Key:') or '\\Q:' in lines[j]):
                        context_text += ' ' + lines[j].strip()
                        j += 1
            
            # Set context if found
            if context_text:
                context = context_text
            
            # Look for Key line
            for line in lines:
                if line.startswith('Key:'):
                    key_text = line.replace('Key:', '').strip()
                    
                    # Extract retrieval_context - the last parentheses
                    source_match = re.search(r'\((.*?)\)$', key_text)
                    if source_match:
                        retrieval_context = source_match.group(1).strip()
                        # Remove the parentheses from the key text to get expected_output
                        expected_output = key_text[:source_match.start()].strip()
                    else:
                        expected_output = key_text
        
        # Create data entry
        entry = {
            "file_path": "example.json",
            "input_key_name": "query",
            "question_number": question_number,
            "question": question,
            "expected_output_key_name": "expected_output",
            "expected_output": expected_output,
            "context_key_name": "context",
            "context": context,
            "retrieval_context_key_name": "retrieval_context",
            "retrieval_context": retrieval_context
        }
        
        data.append(entry)
    
    return data

def save_to_csv(data, output_file="membrane_questions.csv"):
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

def save_to_json(data, output_file="membrane_questions.json"):
    # Create proper JSON structure with the requested fields
    formatted_data = []
    for item in data:
        formatted_item = {
            "query": item["question"],
            "expected_output": item["expected_output"],
            "context": item["context"] if item["context"] else "",
            "retrieval_context": item["retrieval_context"] if item["retrieval_context"] else ""
        }
        formatted_data.append(formatted_item)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(formatted_data, f, ensure_ascii=False, indent=4)
    print(f"Data saved to {output_file}")

# Example usage:
# my_list = [
#     '1\\Q:请用一句话阐述相转化膜的定义。 (通过适当途径使聚合物从均相铸膜液中沉析，形成聚合物富相和聚合物贫相的膜) (GB/T 20103-2006)',
#     '3\\Q:请用一句话阐述复合膜的定义。 (用两种不同的膜材料，分别制成具有分离功能的表面活性层和起支撑作用的多孔层组成的膜) (GB/T 20103-2006)',
#     '16\\Q: How to confirm the successful incorporation of nanoparticles?',
#     'Context: Both TFC and TFN membranes exhibit nano-scale surface roughness...',
#     'Key: In TEM images, they can be shown in a darker color within the cross-section in the membrane. (Journal of Membrane Science 294(2007)1)'
# ]

# parsed_data = parse_questions(my_list)
# save_to_csv(parsed_data)
# save_to_json(parsed_data)

In [50]:
question_list = nonqa_set
parsed_data = parse_questions(question_list)
save_to_json(parsed_data)
save_to_csv(parsed_data)

Data saved to membrane_questions.json
Data saved to membrane_questions.csv


In [51]:
questions = json.load(open("membrane_questions.json", "r"))

In [53]:
save_to_csv(questions)

Data saved to membrane_questions.csv


In [ ]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM

class OpenrouterAPI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = OpenrouterAPI(
    model="claude-3-opus-20240229"
    azure_endpoint=azure_endpoint,
    openai_api_key=openai_api_key,
)
azure_openai = AzureOpenAI(model=custom_model)
print(azure_openai.generate("Write me a joke"))

In [54]:
from deepeval.dataset import EvaluationDataset
from deepeval import evaluate

dataset = EvaluationDataset()
dataset.pull(
  alias="MembraneTest", 
  # True if dataset is pre-computed
  auto_convert_goldens_to_test_cases=False
)

/Users/masa/miniforge3/envs/torch/lib/python3.8/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [56]:
from deepeval.metrics import HallucinationMetric, AnswerRelevancyMetric
from deepeval.dataset import EvaluationDataset

hallucination_metric = HallucinationMetric(threshold=0.3)
answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
evaluate(dataset, [hallucination_metric, answer_relevancy_metric])



In [ ]:
deepeval.login_with_confident_api_key("od9egJUwzwFdCJ6kzxMCrEWpZVsVZtdxsUETSxjausk=")